In [8]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization,Dropout,ActivityRegularization
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.regularizers import l1,l2
import numpy as np


op=[Adam,
    RMSprop]
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train[:50000], x_train[50000:]
y_train, y_test = y_train[:50000], y_train[50000:]
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Function to create and train a model
def train_model(model, optimizer, name):
    print(f"Training {name} model...")
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=64, epochs=5, validation_data=(x_test, y_test))
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(optimizer,f'Test loss ({name}): {test_loss:.4f}, Test accuracy ({name}): {test_accuracy:.4f}')
    print()

    # Plot accuracy and loss curves
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    return history


In [9]:
total_training_images = x_train.shape[0]
total_testing_images = x_test.shape[0]
print("Total training images",total_training_images,"Total testing images",total_testing_images)

Total training images 50000 Total testing images 10000


In [10]:
import numpy as np

def print_image_counts_per_class(x_train, y_train):

  class_counts = {}
  for i in range(len(x_train)):
    class_label = np.argmax(y_train[i])
    if class_label not in class_counts:
      class_counts[class_label] = 0
    class_counts[class_label] += 1

  print('Number of images per class:')
  for class_label, count in class_counts.items():
    print(f'Class {class_label}: {count}')

# Print the number of images per class for the MNIST training dataset
print_image_counts_per_class(x_train, y_train)


Number of images per class:
Class 5: 4506
Class 0: 4932
Class 4: 4859
Class 1: 5678
Class 9: 4988
Class 2: 4968
Class 3: 5101
Class 6: 4951
Class 7: 5175
Class 8: 4842


In [ ]:
# create and train customized CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])
for i in op:
  train_model(model, i(learning_rate=0.01), "Customized CNN Model")


In [ ]:
model.summary()

In [ ]:
# create and train customized CNN (L1 Regularization)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), kernel_regularizer=l1(0.001)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l1(0.001)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l1(0.001)),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l1(0.001)),
    Dense(10, activation='softmax')
])
for i in op:
  train_model(model, i(learning_rate=0.01), "Customized CNN Model")

In [ ]:
# create and train customized CNN (L2 Regularization)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(10, activation='softmax')
])
for i in op:
  train_model(model, i(learning_rate=0.01), "Customized CNN Model")

In [ ]:
# create and train customized CNN (Drop Regularization)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
    Dense(10, activation='softmax')
])
for i in op:
  train_model(model, i(learning_rate=0.01), "Customized CNN Model")

In [ ]:
# Create and train the LeNet-5 model
lenet5_model = Sequential([
    Conv2D(6, (5, 5), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(16, (5, 5), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(10, activation='softmax')
])

for i in op:
  train_model(lenet5_model, i(learning_rate=0.01), "LeNet-5")

In [ ]:
# Create and train a simplified ResNet-like model
resnet_like_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')
])
for i in op:
  train_model(resnet_like_model, i(learning_rate=0.001), "ResNet-like")

In [ ]:
vgg16_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])
for i in op:
  train_model(vgg16_model, i(learning_rate=0.001), "VGG16")